In [1]:
# get current directory and append system path
import os
import sys
import json
import numpy as np
import glob
import tqdm

In [2]:
path = os.getcwd()
# go up one directory
path = os.path.dirname(path)
sys.path.append(path)
# go to the following:
# classification/pointnetpp/data/buildings_split
data_load_path = os.path.join(path, 'classification', 'pointnetpp', 'data', 'buildings_split')
data_save_path = os.path.join(path, 'densification', 'sapcu', 'buildings_split')
# This path contains a bunch of .npy files
# we need to do some processing on them

In [3]:
# The processing we will do:
# 1. Load the .npy files
# 2. For each .npy file, we will:
#   a. Extract only xyz (first 3 columns)
#   b. Convert to .xyz file
#   c. Save to a new directory (the save path)

In [4]:
# load the .npy files
min_coords = {}
files = glob.glob(os.path.join(data_load_path, '*.npy'))
for file in tqdm.tqdm(files):
    # do the processing only if it ends on _points.npy
    if not file.endswith('_points.npy'):
        continue
    # load the file
    points = np.load(file)
    # extract only xyz
    xyz = points[:, :3]
    # find the corresponding _labels.npy file
    labels_file = file[:-11] + '_labels.npy'
    # load the labels
    labels = np.load(labels_file)
    # convert labels to int
    labels = labels.astype(int)
    # filter the points to only include ones that have a label of 5 (building)
    xyz = xyz[labels == 5]
    # convert to double precision float
    xyz = xyz.astype(np.float64)
    # save the min x, y, z in the dictionary
    # get the file name
    file_name = os.path.basename(file)
    # remove the .npy extension
    file_name = file_name[:-4]
    # add the .xyz extension
    file_name = file_name + '.xyz'
    # save to the new directory
    if not os.path.exists(data_save_path):
        os.makedirs(data_save_path)
    np.savetxt(os.path.join(data_save_path, file_name), xyz, delimiter=' ', fmt='%1.8e')

# save the dictionary with the min x, y, z as a json file

with open(os.path.join(data_save_path, 'min_coords.json'), 'w') as f:
    json.dump(min_coords, f)
    
print('Done')

100%|██████████| 864/864 [00:39<00:00, 21.77it/s]

Done
